In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime
import time
tqdm.tqdm.pandas()

In [2]:
bulk_df = pd.read_excel('./공공주택찾기/인천/일괄계약전체_20180101_20220224.xlsx')
print(bulk_df.shape)
bulk_df.head()

(4686, 15)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식,부동산유형,일괄계약
0,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,67.94,14140,4,2018,2018-12-31,208.124816,4,연립다세대,Y
1,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,67.94,14140,3,2018,2018-12-31,208.124816,4,연립다세대,Y
2,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,67.94,14140,2,2018,2018-12-31,208.124816,4,연립다세대,Y
3,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,68.59,14275,5,2018,2018-12-31,208.120717,4,연립다세대,Y
4,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,69.09,14379,4,2018,2018-12-31,208.119844,4,연립다세대,Y


In [3]:
bulk_df['소유자'] = np.nan

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST driver version for 98.0.4758
Get LATEST driver version for 98.0.4758
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\max6296\.wdm\drivers\chromedriver\win32\98.0.4758.102]


In [6]:
driver.get("http://www.iros.go.kr/PMainJ.jsp")

In [7]:
driver.get("http://www.iros.go.kr/frontservlet?cmd=RISUWelcomeViewC")

In [8]:
def get_owner_name(df):
    df = df.copy()
    
    count = 0
    for addr in tqdm.tqdm_notebook(df['지번주소'].unique()):
        owner_name = np.nan        
        addr_df = df[df['지번주소'] == addr]
        addr = addr.replace('-0', '')
        
        driver.switch_to.default_content()
        
        
        driver.switch_to.frame("inputFrame")
        
        elem_search = driver.find_element_by_xpath('html/body/form[1]/div[24]/div/div/div/fieldset/div/table/tbody/tr[5]/td/span/input')
        elem_search.clear()
        elem_search.send_keys(addr)
        elem_search.send_keys(Keys.RETURN)
        time.sleep(2)
        
        try:
            driver.switch_to.alert.accept()
        except:
            pass
        
        time.sleep(1)
        
        driver.switch_to.default_content()
        driver.switch_to.frame('resultFrame')
        driver.switch_to.frame('frmOuterModal')
        
        try:
            driver.find_element_by_xpath('/html/body/div[2]/div[2]/table/tbody/tr[3]/td[6]/button').click()
        except:
            continue
        
        time.sleep(1)
        
        elem_result = driver.find_element_by_xpath('/html/body/div[2]/div[2]/table/tbody/tr/td[3]')
        
        owner_name = elem_result.text
        
        driver.switch_to.default_content()
        
        df.loc[addr_df.index, '소유자'] = owner_name
        
        count += 1
    
    return df

In [9]:
prac_df = get_owner_name(bulk_df)

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/246 [00:00<?, ?it/s]

In [10]:
print(prac_df.shape)
prac_df.head()

(4686, 16)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식,부동산유형,일괄계약,소유자
0,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,67.94,14140,4,2018,2018-12-31,208.124816,4,연립다세대,Y,한***~
1,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,67.94,14140,3,2018,2018-12-31,208.124816,4,연립다세대,Y,한***~
2,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,67.94,14140,2,2018,2018-12-31,208.124816,4,연립다세대,Y,한***~
3,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,68.59,14275,5,2018,2018-12-31,208.120717,4,연립다세대,Y,한***~
4,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,69.09,14379,4,2018,2018-12-31,208.119844,4,연립다세대,Y,한***~


In [11]:
prac_df['소유자'].value_counts()

한***~        2397
인***~        1169
서**           200
주식회사무***~     104
박**            85
권**            77
이**            71
김**            61
오**            60
정**            48
남**            40
정**외 1명        35
노**            33
오**외 1명        28
임**            25
주**            24
김**외 1명        24
박**외 1명        19
주식회사지***       19
한**            16
대***~          14
국***~          14
주식회사생***~      12
전**            12
최**외 1명        11
주식회사제***~      11
조**외 1명        10
류**외 1명        10
Name: 소유자, dtype: int64

In [12]:
prac_df['소유자'].isna().sum()

57

In [13]:
prac_df = prac_df.dropna(subset=['소유자']).reset_index(drop=True)

In [14]:
prac_df.shape

(4629, 16)

In [15]:
def sh_lh(x):
    if x == '한***~':
        return 'LH'
    else:
        return x

In [16]:
prac_df['소유자'] = prac_df['소유자'].apply(sh_lh)

In [17]:
prac_df['소유자'].value_counts()

LH           2397
인***~        1169
서**           200
주식회사무***~     104
박**            85
권**            77
이**            71
김**            61
오**            60
정**            48
남**            40
정**외 1명        35
노**            33
오**외 1명        28
임**            25
주**            24
김**외 1명        24
박**외 1명        19
주식회사지***       19
한**            16
대***~          14
국***~          14
주식회사생***~      12
전**            12
최**외 1명        11
주식회사제***~      11
조**외 1명        10
류**외 1명        10
Name: 소유자, dtype: int64

In [18]:
%%time
prac_df.to_excel('./공공주택찾기/인천/일괄계약_전체_소유자포함_20180101_20220224.xlsx', index=False)

Wall time: 1.37 s


In [19]:
prac_df = prac_df[
    (prac_df['소유자'] == 'LH')
]
print(prac_df.shape)
prac_df.head()

(2397, 16)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식,부동산유형,일괄계약,소유자
0,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,67.94,14140,4,2018,2018-12-31,208.124816,4,연립다세대,Y,LH
1,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,67.94,14140,3,2018,2018-12-31,208.124816,4,연립다세대,Y,LH
2,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,67.94,14140,2,2018,2018-12-31,208.124816,4,연립다세대,Y,LH
3,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,68.59,14275,5,2018,2018-12-31,208.120717,4,연립다세대,Y,LH
4,인천광역시 강화군 강화읍 관청리 555-4,북문길9번길 21,인천광역시 강화군 강화읍 관청리,555,4,송학하얀집1동2동,69.09,14379,4,2018,2018-12-31,208.119844,4,연립다세대,Y,LH


In [20]:
prac_df['소유자'].value_counts()

LH    2397
Name: 소유자, dtype: int64

In [21]:
%%time
prac_df.to_excel('./공공주택찾기/인천/LH_거래사례_20180101_20220224.xlsx', index=False)

Wall time: 685 ms
